In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Download the relevant data
import os
from langchain_community.utilities import SQLDatabase

import requests
import os

url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
filename = "./data/chinook/Chinook_Sqlite.sqlite"

folder = os.path.dirname(filename)

if not os.path.exists(folder):
    os.makedirs(folder)

if not os.path.exists(filename):
    response = requests.get(url)
    with open(filename, "wb") as file:
        file.write(response.content)
    print("Chinook database downloaded")

db = SQLDatabase.from_uri(f"sqlite:///{filename}")

Chinook database downloaded


In [3]:
from opik.integrations.openai import track_openai
from openai import OpenAI
import json

os.environ["OPIK_PROJECT_NAME"] = "langchain-integration-demo"
client = OpenAI()

openai_client = track_openai(client)

prompt = """
Create 20 different example questions a user might ask based on the Chinook Database.

These questions should be complex and require the model to think. They should include complex joins and window functions to answer.

Return the response as a json object with a "result" key and an array of strings with the question.
"""

completion = openai_client.chat.completions.create(
    model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}]
)

print(completion.choices[0].message.content)

OPIK: Started logging traces to the "langchain-integration-demo" project at http://trainingvm:5173/api/v1/session/redirect/projects/?trace_id=0195c7ed-5aae-77cd-82b8-571bdc32664f&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv.


{
    "result": [
        "Which customer has spent the most money on purchases?",
        "What is the average total price of all invoices for customers from the USA?",
        "Which genre has the highest total sales revenue?",
        "How many tracks belong to the genre 'Rock' and were purchased by customers from the USA?",
        "What is the average number of tracks in each playlist?",
        "Which customer has the highest total number of purchased tracks?",
        "What is the total revenue from digital purchases for customers from Canada?",
        "Which employee has the highest total sales revenue?",
        "How many customers have made more than 10 purchases?",
        "What is the average total price of invoices for each employee?",
        "Which artist has the highest total sales revenue?",
        "What is the average total time of all playlists?",
        "How many tracks were sold in each country?",
        "What is the total revenue from all purchases in the year

In [4]:
# Create the synthetic dataset
import opik

synthetic_questions = json.loads(completion.choices[0].message.content)["result"]

client = opik.Opik()

dataset = client.get_or_create_dataset(name="synthetic_questions")
dataset.insert([{"question": question} for question in synthetic_questions])

OPIK: Created a "synthetic_questions" dataset at http://trainingvm:5173/api/v1/session/redirect/datasets/?dataset_id=0195c7ee-0c53-7384-adae-905a39aa4f68&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv.


In [5]:
# Use langchain to create a SQL query to answer the question
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from opik.integrations.langchain import OpikTracer

opik_tracer = OpikTracer(tags=["simple_chain"])

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db).with_config({"callbacks": [opik_tracer]})
response = chain.invoke({"question": "How many employees are there ?"})
response

print(response)

SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM "Employee"


In [6]:
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import base_metric, score_result
from typing import Any


class ValidSQLQuery(base_metric.BaseMetric):
    def __init__(self, name: str, db: Any):
        self.name = name
        self.db = db

    def score(self, output: str, **ignored_kwargs: Any):
        # Add you logic here

        try:
            db.run(output)
            return score_result.ScoreResult(
                name=self.name, value=1, reason="Query ran successfully"
            )
        except Exception as e:
            return score_result.ScoreResult(name=self.name, value=0, reason=str(e))


valid_sql_query = ValidSQLQuery(name="valid_sql_query", db=db)

client = Opik()
dataset = client.get_dataset("synthetic_questions")


@track()
def llm_chain(input: str) -> str:
    response = chain.invoke({"question": input})

    return response


def evaluation_task(item):
    response = llm_chain(item["question"])

    return {"output": response}


res = evaluate(
    experiment_name="SQL question answering",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[valid_sql_query],
    nb_samples=20,
)

Evaluation: 100%|██████████| 20/20 [00:05<00:00,  3.78it/s]


╭─ synthetic_questions (20 samples) ─╮
│                                    │
│ Total time:        00:00:05        │
│ Number of samples: 20              │
│                                    │
│ valid_sql_query: 0.8000 (avg)      │
│                                    │
╰────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=791513;http://trainingvm:5173/api/v1/session/redirect/experiments/?experiment_id=0195c7ef-479f-7589-8cac-798d8c449cb1&dataset_id=0195c7ee-0c53-7384-adae-905a39aa4f68&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv\in your Opik dashboard]8;;\.